In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [3]:
system_message = "You are a helpful assistant for an Airline called Flight AIrline. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
#Price function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function}]

In [17]:
#Discount function

discount_prices = {"london": "$10", "paris": "$7"}

def get_discount_price(destination_city):
    print(f"Tool get_discount_price called for {destination_city}")
    city = destination_city.lower()
    return discount_prices.get(city, 0)

In [19]:
discount_function = {
    "name": "get_discount_price",
    "description": "Get the discount of a ticket to the destination city. Call this whenever you need to know the discount price, for example when a customer asks 'how much is the discount for a trip to london'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to know about discount",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [20]:
tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": discount_function}]

In [30]:
#Airplane function

airplane = {"london": "A380", "paris": "Boeing 770"}

def get_airplane(destination_city):
    print(f"Tool get_airplane called for {destination_city}")
    city = destination_city.lower()
    return airplane.get(city, "Check at the airport")

In [32]:
airplane_function = {
    "name": "get_airplane",
    "description": "Get the airplane of a destination city. Call this whenever you need to know about the airplane used for some destination, for example when a customer asks 'What model of plane will be used for this trip?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The airplane that the customer wants to know about",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [33]:
tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": discount_function},
        {"type": "function", "function": airplane_function}]

## Getting OpenAI to use Tools

In [34]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    
    if tool_call.function.name == "get_ticket_price":
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
    elif tool_call.function.name == "get_discount_price":
        discount = get_discount_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"discount": discount}),
            "tool_call_id": message.tool_calls[0].id
        }
    elif tool_call.function.name == "get_airplane":
        airplane = get_airplane(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"airplane": airplane}),
            "tool_call_id": message.tool_calls[0].id
        }
        
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()